In [3]:
import pandas as pd
data = pd.read_excel('./电力现货市场QA.xlsx',header=None)
questions = data.loc[1:,0]
answers = data.loc[1:,1]


In [4]:
# 问题与答案
powermarketQAs = [(q,a) for q,a in zip(questions,answers)]
powermarketQAs[124]

('125.电力现货技术支持系统与电力交易平台定位与关系是什么？',
 '随着中发9号文的发布，我国电力市场改革不断深化，电力交易品种日益丰富，以中长期交易规避风险，以现货市场发现价格的“中长期+现货”联合运营模式逐渐形成。中发9号文配套文件《关于推进电力市场建设的实施意见》指出，电力市场主要由中长期市场和现货市场构成，中长期市场主要开展多年、年、季、月、周等日以上电能量交易和可中断负荷、调压等辅助服务交易。现货市场主要开展日前、日内、实施电能量交易和备用、调频等辅助服务交易。条件成熟时，探索开展容量市场、电力期货和衍生品等交易。2018年，国家电网公司《全国统一电力市场深化设计方案》在综合考虑体制架构和电网安全运行需要的基础上，提出了由交易中心负责市场注册、交易申报、交易结算、信息发布等与市场交易密切相关环节，由调控中心负责现货和辅助服务交易组织、安全校核等与电网运行密切相关环节的统一市场联合运营模式。此模式也成为了电力市场技术支撑平台在交易中心和调控中心之间功能建设的实际划分标准，交易中心的电力交易平台、调控中心的电力现货技术支持系统各功能相互协作实现“全时段、全过程、全环节”的交易流程，实现“中长期+现货”市场安全、高效、有序运营，向市场成员提供准确规范的市场交易信息和在线交易业务支持。调控中心负责现货交易与电网运行密切相关的环节，负责安全校核、现货交易组织、辅助服务交易组织等工作。电力现货技术支持系统面向调控中心业务，用于支撑日前、实时等电力现货市场的组织、出清、安全校核等核心业务，并与调控中心内部的调度系统、交易中心的电力交易平台等系统实现对接。交易中心负责与市场交易密切相关的环节，电力交易平台作为国网公司对市场主体服务的窗口，在市场注册、交易申报、交易结算、信息发布等环节为市场主体提供相关服务，并与包括电力现货技术支持系统在内的调度、营销、财务等系统实现对接。电力交易平台分为面向电力交易中心的主站端，以及面向发电侧、用电侧等市场成员的子站端。主站端的主要功能是市场成员注册审批、中长期交易组织、政府授权合约分解、合同管理、结算计算、信息发布、信息管理、信用管理等功能；子站端是发用侧市场成员用于进行市场注册、现货市场申报、中长期市场申报、中长期合约备案、查看发布信息、查看结算账单的系统。分主要交易环节看：（1）市场成员管理：市场成员向电力交易平台提交注册申请

### Step1 导入相关包

In [5]:
import langsmith
from langchain import chat_models, prompts, smith
from langchain.schema import output_parser
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
import bs4
from langchain import hub
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document
from langchain_community.llms import BaichuanLLM
from langchain_community.vectorstores import FAISS

USER_AGENT environment variable not set, consider setting it to identify your requests.


### 索引

In [6]:
QA_list = []
for q,a  in zip(questions,answers):
    qa = q + '\n' + a
    QA_list.append(qa)

print(QA_list[0])

1.什么是电力市场？
电力市场的概念。我国关于电力市场的权威解释始见于《中国电力百科全书电力系统卷（第二版）》。电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现电力商品交换的电力工业组织结构、经营管理和运行规则的总和。电力市场又是一个具体的执行系统，包括交易场所、交易管理系统、计量和结算系统、信息和通信系统等。上面从组织和实操两个维度对电力市场进行了描绘。通常电力市场包括广义和狭义两种含义。比照商品市场的一般定义，广义的电力市场泛指电力流通交换的领域。按照该含义，自电力作为商品实现交换之日起，电力市场就已经存在。当今在涉及市场规模和范围的语境下，运用的便是广义电力市场的概念。广义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范围的电网才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。狭义的电力市场是指现代竞争性的电力市场，《中国电力百科全书》定义的电力市场即为狭义的电力市场，建立电力市场旨在通过开放、竞争等市场手段实现电力能源资源的优化配置。所谓基于市场经济原则主要是指电能生产者和使用者本着公平竞争、自愿互利的原则，通过协商、竞价等方式，就电能及其相关产品进行交易，通过市场竞争确定价格和数量的市场原则。目前广泛讨论的电力市场，除特殊语境下，通常都是指狭义的电力市场。之所以称之为狭义的电力市场，主要原因有二。1）专指现代电力市场，狭义的电力市场兴起于20世纪80年代，是在电力作为商品出现的百年之后，而这百年间，广义电力市场的形态发生了曲折的变迁。电力工业初创期的电力领域是以残酷的市场竞争形态出现的，既有生产方式的竞争（交流与直流之争），也有垂直一体化模式下的电力企业之间抢占市场范围和争夺同一服务地域内的用户之争。这种竞争，一方面有力推动了电力工业的技术进步；另一方面，却也限制了规模经济效益的发挥，并因重复建设等过度竞争而造成资源浪费。因此，20世纪初，电力业态很快从无序竞争状态演变为政府管制或企业自律下的垄断市场形态，尤其是在二次世界大战之后，鉴于恢复经济的需要，大多数国家都采取国家垄断的模式。直至20世纪末，随着电网的发展和技术的进步，电力规模经济的低垂之果几

In [7]:
# 将分割后的文本块封装成Document类
docs = []
for i in range(0,len(QA_list)):
    qa_doc = Document(page_content=QA_list[i], metadata={"source": "PowerMarketQA\电力现货市场101问.xlsx","order":i+1})
    docs.append(qa_doc)

docs[0]

Document(metadata={'source': 'PowerMarketQA\\电力现货市场101问.xlsx', 'order': 1}, page_content='1.什么是电力市场？\n电力市场的概念。我国关于电力市场的权威解释始见于《中国电力百科全书电力系统卷（第二版）》。电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现电力商品交换的电力工业组织结构、经营管理和运行规则的总和。电力市场又是一个具体的执行系统，包括交易场所、交易管理系统、计量和结算系统、信息和通信系统等。上面从组织和实操两个维度对电力市场进行了描绘。通常电力市场包括广义和狭义两种含义。比照商品市场的一般定义，广义的电力市场泛指电力流通交换的领域。按照该含义，自电力作为商品实现交换之日起，电力市场就已经存在。当今在涉及市场规模和范围的语境下，运用的便是广义电力市场的概念。广义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范围的电网才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。狭义的电力市场是指现代竞争性的电力市场，《中国电力百科全书》定义的电力市场即为狭义的电力市场，建立电力市场旨在通过开放、竞争等市场手段实现电力能源资源的优化配置。所谓基于市场经济原则主要是指电能生产者和使用者本着公平竞争、自愿互利的原则，通过协商、竞价等方式，就电能及其相关产品进行交易，通过市场竞争确定价格和数量的市场原则。目前广泛讨论的电力市场，除特殊语境下，通常都是指狭义的电力市场。之所以称之为狭义的电力市场，主要原因有二。1）专指现代电力市场，狭义的电力市场兴起于20世纪80年代，是在电力作为商品出现的百年之后，而这百年间，广义电力市场的形态发生了曲折的变迁。电力工业初创期的电力领域是以残酷的市场竞争形态出现的，既有生产方式的竞争（交流与直流之争），也有垂直一体化模式下的电力企业之间抢占市场范围和争夺同一服务地域内的用户之争。这种竞争，一方面有力推动了电力工业的技术进步；另一方面，却也限制了规模经济效益的发挥，并因重复建设等过度竞争而造成资源浪费。因此，20世纪初，电力业态很快从无序竞争状态演变为

In [8]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="moka-ai/m3e-base",  # 使用m3e模型做embeddding
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
vectorstore = FAISS.from_documents(docs, huggingface_embeddings)
retriever = vectorstore.as_retriever()

d:\Download\miniconda3\envs\py3.9\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 构建问答链

In [12]:
from langchain.prompts import ChatPromptTemplate
template = """请使用以下检索到的背景信息来回答问题。如果你不知道答案，直接说你不知道。
问题：{question} 

背景：{context} 

答案：
"""

prompt = ChatPromptTemplate.from_template(template)

#llm = chat_models.ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# llm = Ollama(model="qwen2:1.5b", temperature=0)

llm = BaichuanLLM(model='Baichuan3-Turbo-128k')

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [13]:
# 执行查询
rag_chain.invoke("电力市场与普通商品市场的差异？")

'电力市场与普通商品市场的差异主要体现在以下几个方面：\n\n1. **无仓储性**：电力商品的生产、交割和消费几乎是同时完成的，电能的存储成本极高且技术难度大，因此电力市场不像普通商品市场那样存在仓储和物流环节。\n\n2. **同质性**：电能不带有任何生产者的标识，一旦进入电网，就失去了生产者的个体特征，用户无法区分电能的来源，这使得电力市场上的电能具有高度的同质性。\n\n3. **可预测性**：电力需求在较长周期内会呈现周期性波动，这与普通商品市场的波动性有所不同，电力市场的这种特性在一定程度上抑制了投机行为，但也增加了市场主体滥用市场力的可能性。\n\n4. **双重属性**：电力商品具有生产资料和生活资料的双重属性，它既关系到国家的经济发展，又关系到民众的生活质量。\n\n5. **现货市场的特殊性**：电力现货市场需要维持实时平衡，因为电能的生产和消费必须即时匹配。此外，电力传输服从基尔霍夫定律，这意味着在电网输送能力有限的情况下，不是所有的交易都能顺利完成交割。电力现货市场还需要耦合辅助服务市场，因为电能量的生产和辅助服务是相互制约、相互依存的。最后，电力的快速传播性使得用户享有的产品具有同质性，同时也要求电力市场必须共同维护电网的安全和优质。\n\n这些特殊性导致电力市场在组织结构、经营管理和运行规则上与普通商品市场有显著差异，需要专门的市场规则和协议来确保市场的有效运作。'

In [2]:
from langchain_community.llms import BaichuanLLM
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from typing import List
from sentence_transformers import CrossEncoder

d:\Download\miniconda3\envs\py3.9\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step2 加载数据

In [2]:
import pandas as pd

In [3]:
data = pd.read_excel('./电力现货市场QA.xlsx',header=None)
questions = data.loc[1:,0]
answers = data.loc[1:,1]

In [4]:
check_inputs = {}
results = []

In [5]:
QA_list = []
for index , (q , a)in enumerate(zip(questions,answers)):
    qa = q + '\n' + a
    results.append({"query_id":index+1,
                    "query":q,
                    "gt_answer":a
                    })
    QA_list.append(qa)


In [6]:
print(results[0])

{'query_id': 1, 'query': '1.什么是电力市场？', 'gt_answer': '电力市场的概念。我国关于电力市场的权威解释始见于《中国电力百科全书电力系统卷（第二版）》。电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现电力商品交换的电力工业组织结构、经营管理和运行规则的总和。电力市场又是一个具体的执行系统，包括交易场所、交易管理系统、计量和结算系统、信息和通信系统等。上面从组织和实操两个维度对电力市场进行了描绘。通常电力市场包括广义和狭义两种含义。比照商品市场的一般定义，广义的电力市场泛指电力流通交换的领域。按照该含义，自电力作为商品实现交换之日起，电力市场就已经存在。当今在涉及市场规模和范围的语境下，运用的便是广义电力市场的概念。广义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范围的电网才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。狭义的电力市场是指现代竞争性的电力市场，《中国电力百科全书》定义的电力市场即为狭义的电力市场，建立电力市场旨在通过开放、竞争等市场手段实现电力能源资源的优化配置。所谓基于市场经济原则主要是指电能生产者和使用者本着公平竞争、自愿互利的原则，通过协商、竞价等方式，就电能及其相关产品进行交易，通过市场竞争确定价格和数量的市场原则。目前广泛讨论的电力市场，除特殊语境下，通常都是指狭义的电力市场。之所以称之为狭义的电力市场，主要原因有二。1）专指现代电力市场，狭义的电力市场兴起于20世纪80年代，是在电力作为商品出现的百年之后，而这百年间，广义电力市场的形态发生了曲折的变迁。电力工业初创期的电力领域是以残酷的市场竞争形态出现的，既有生产方式的竞争（交流与直流之争），也有垂直一体化模式下的电力企业之间抢占市场范围和争夺同一服务地域内的用户之争。这种竞争，一方面有力推动了电力工业的技术进步；另一方面，却也限制了规模经济效益的发挥，并因重复建设等过度竞争而造成资源浪费。因此，20世纪初，电力业态很快从无序竞争状态演变为政府管制或企业自律下的垄断市场形态，尤其是在二次世界大战之后，鉴于恢复经济的需要，大多数国家都采取

In [7]:
# 将分割后的文本块封装成Document类
docs_after_split = []
for i in range(0,len(QA_list)):
    qa_doc = Document(page_content=QA_list[i], metadata={"source": "PowerMarketQA\电力现货市场101问.xlsx","order":i+1})
    docs_after_split.append(qa_doc)

docs_after_split[0]

Document(metadata={'source': 'PowerMarketQA\\电力现货市场101问.xlsx', 'order': 1}, page_content='1.什么是电力市场？\n电力市场的概念。我国关于电力市场的权威解释始见于《中国电力百科全书电力系统卷（第二版）》。电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现电力商品交换的电力工业组织结构、经营管理和运行规则的总和。电力市场又是一个具体的执行系统，包括交易场所、交易管理系统、计量和结算系统、信息和通信系统等。上面从组织和实操两个维度对电力市场进行了描绘。通常电力市场包括广义和狭义两种含义。比照商品市场的一般定义，广义的电力市场泛指电力流通交换的领域。按照该含义，自电力作为商品实现交换之日起，电力市场就已经存在。当今在涉及市场规模和范围的语境下，运用的便是广义电力市场的概念。广义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范围的电网才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。狭义的电力市场是指现代竞争性的电力市场，《中国电力百科全书》定义的电力市场即为狭义的电力市场，建立电力市场旨在通过开放、竞争等市场手段实现电力能源资源的优化配置。所谓基于市场经济原则主要是指电能生产者和使用者本着公平竞争、自愿互利的原则，通过协商、竞价等方式，就电能及其相关产品进行交易，通过市场竞争确定价格和数量的市场原则。目前广泛讨论的电力市场，除特殊语境下，通常都是指狭义的电力市场。之所以称之为狭义的电力市场，主要原因有二。1）专指现代电力市场，狭义的电力市场兴起于20世纪80年代，是在电力作为商品出现的百年之后，而这百年间，广义电力市场的形态发生了曲折的变迁。电力工业初创期的电力领域是以残酷的市场竞争形态出现的，既有生产方式的竞争（交流与直流之争），也有垂直一体化模式下的电力企业之间抢占市场范围和争夺同一服务地域内的用户之争。这种竞争，一方面有力推动了电力工业的技术进步；另一方面，却也限制了规模经济效益的发挥，并因重复建设等过度竞争而造成资源浪费。因此，20世纪初，电力业态很快从无序竞争状态演变为

### Step3 创建向量数据库

In [8]:
# 从过往工作经验看，embedding对于rag效果影响比较大，一般首选还是openai embedding做这一部分，开源的效果很一般
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="moka-ai/m3e-base",  # 使用m3e模型做embeddding
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

In [9]:
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)

### Step4 创建QA链

In [10]:
query = """输配电价对现货市场有什么影响？"""
         # Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query,k=3)
for i, doc in enumerate(relevant_documents):
    print(f"检索到的第{i+1}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")


# 从这一步看，已经很清晰rag的原理了，即从大量文件索引找到top k相关的text块，供下一步LLM查找、总结答案

检索到的第1个内容: 
 34.输配电价对现货市场有什么影响？
输配电价机制对电力现货市场的影响。输配电价机制是电力市场机制的重要部分，同样也是影响电力现货市场的重要因素。一般来说，电力现货市场的设计目标是理顺电力价格的形成机制，提高能源利用效率，降低电力成本，促进节能环保。电力现货市场设计应能够通过发现完整的电力价格信号，引导市场主体开展电力交易，为资源优化配置、规划投资提供可量化的参考依据，实现电力市场交易与电力系统实际运行的高效匹配。其核心是发现完整的电力价格信号。输配电价机制会影响电力现货市场设计，影响参与现货市场的市场主体的收益，进而会影响电力现货市场运行，主要表现在以下方面。1）输配电价机制影响电力现货市场设计。电价形成是电力市场的核心，涉及发电厂商、电网企业以及电力用户的利益。输配电价是电力产业价格链的中间环节，合理的输配电价机制是电力现货市场的基础和关键，它需要反映电力生产的真实成本和市场供需关系，从而为各市场成员提供明确的价格信号，正确引导市场消费和电力投资，优化社会资源的配置。2）输配电价机制影响参与现货市场的市场主体的收益。电价体系分为上网电价、输配电价和销售电价，在市场化的电力体制中，上网电价和销售电价由竞争产生，而输配电价往往由监管机构进行独立核定。但输配电价形成和分摊会影响市场主体参与现货市场的竞争方式，进而影响现货市场的市场主体的收益。3）输配电价机制会影响电力现货市场运行。合理的输配电价是电力市场有效运作的必要条件，输配电价空间会受到上网电价和销售电价的制约，如果电网合理成本的回收、合理收益的获取得不到根本保证，会严重地影响电网建设资本金的筹集。合理的输配电费用定价和分摊有利于向电力交易主体提供有效的经济信号，促进电力资源优化配置，并促进电网企业更有效地管理、规划输配电资源。
-----------------------------------------------------
检索到的第2个内容: 
 28.什么是现货市场中的再调度法？
再调度法（re-dispatch）是一种实时消除阻塞的方法，这种阻塞管理方法不直接改变日前市场中标的发电量和价格，而是一个独立、自愿参与的市场，发电机组可以在其日前发电计划的基础上，对向上或向下调整的发电量进行报价，即机组的上调、下调报价，并在实际调用后获得收益。（1）再调度法原理。系统运营商作

In [24]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder(
    "BAAI/bge-reranker-base", max_length=512, device="cpu"
)
reranked_docs = cross_encoder.rank(
        query,
        [doc.page_content for doc in relevant_documents],
        top_k=2,
        return_documents=True,)
reranked_docs

[{'corpus_id': 2,
  'score': 0.99910444,
  'text': '19.输配电价的主要定价方法是什么？输配电价对现货市场有什么影响？我国输配电价的形成机制和执行方式是什么？\n（1）输配电价的主要定价方法。输配电价体系设计、定价方法的选择很大程度与电网的技术和经济特性相关，并应考虑电力改革的实际需求。由于各国国情不同，电力工业改革进程千差万别，各国输配电价制定方法各有不同。输配电网具有自然垄断特性，垄断将造成市场机制失灵，必须通过政府管制制定输配电价，这是世界各国理论和实践中达成的共识。国际上，输配电费用的计算可分为会计成本法和边际成本法两类，前者根据电网过去发生的投资成本和运行成本计算输配电费用，后者根据提供输配电引起输配电网未来投资成本的微增变化计算输配电费用。1）会计成本法。分为以下几类：a）邮票法。按整个电网输送的电量或功率平均分摊整个电网的输配电成本，与输送电能距离、流入流出节点位置无关的方法。该方法首先考虑各部分特定输电设备的成本和电网运行维护费用，形成输电总成本后，再按输电功率计算输电费。这种方法算法简单、直接透明，有利于维持电力交易的同一性和流畅性，同时也降低了独立发电厂投资的风险性。但是由于没有考虑发电资源和输电成本差异，该方法使得市场化前不同价区的发电价格趋同，使市场平均购电价格上扬，不能起到引导电源投资的经济信号作用。b）兆瓦千米法。计算输配电网所有线路和设备的每千瓦每千米的成本，通过潮流计算确定某项输配电服务的实际距离，按输送电能的实际距离和功率计算费用的方法。该方法适用于链条型输电网，对于公共输电网络难于进行合理的成本分摊和定价，也没有很好的考虑交易导致的阻塞成本以及额外的输电损失成本，实际上并不能反映每个交易对输电设备的实际使用度，从而降低了该方法的公正性。c）边界潮流法。先计算电网的简化潮流图，再根据潮流分布按各电压等级功率或电量的来源比例分摊输配电成本的方法。该方法体现了发电资源和输电成本的差异，不会由此造成市场平均购电价格上扬，对相关节点参与现货市场采取何种报价策略有一定影响，进而具有有效引导电源投资的经济信号作用。由于输配电价最终分摊至各项输配电服务，形成服务价格，因此输配电价格机制与电力现货市场交易价格形成机制密切相关。不同输配电价定价方法（如邮票法、兆瓦千米法和边界潮流法）对现货市场有不同

In [11]:
# 创建检索器
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [26]:

""" 使用自定义的检索器,重写_get_relevant_documents方法来实现对相似性检索召回的文本块进行重排"""
class CustomRetriever(BaseRetriever):
    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        cross_encoder = CrossEncoder("BAAI/bge-reranker-base", max_length=512, device="cpu")
        relevant_documents = vectorstore.similarity_search(query,k=5)
        reranked_docs = cross_encoder.rank(
        query,
        [doc.page_content for doc in relevant_documents],
        top_k=2,
        return_documents=True,)

        reranking_relevant_documents= []
        for doc in reranked_docs:
           reranking_relevant_documents.append(
               Document(page_content=doc["text"],
                        metadata=relevant_documents[doc["corpus_id"]].metadata)
                        ) 
        return reranking_relevant_documents

In [27]:
# 自定义加入重排的检索器
retriever = CustomRetriever()

In [12]:
# 创建prompt
"""prompt :请参考下面的相关文本回答问题，一段文本中可能有多个子问题，子问题以问号“？”分隔，子问题对应的答案以小括号“（）”进行分隔，
检索到与问题相关的子问题直接输出子问题对应的答案的全部内容，如果不知道答案，就回复不知道。
"""
prompt_template = """
请用下面相关文本回答问题，如果不知道答案，就回复不知道。
{context}

Question: {question}

Helpful Answer:
"""

prompt = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [13]:
# 创建百川的LLM
llm = BaichuanLLM(model='Baichuan3-Turbo-128k')

In [14]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [15]:
for index,query in enumerate(questions):
    response = qa_chain({"query": query})
    results[index]["response"] = response['result']
    retrieved_docs = []
    for doc in response['source_documents']:
        retrieved_docs.append({"doc_id":doc.metadata['order'],
                               "text":doc.page_content
                               })
    results[index]["retrieved_context"] = retrieved_docs
    print(f"第{index+1}个问题已回复..........")

results[124]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20288\806108324.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"query": query})


第1个问题已回复..........
第2个问题已回复..........
第3个问题已回复..........
第4个问题已回复..........
第5个问题已回复..........
第6个问题已回复..........
第7个问题已回复..........
第8个问题已回复..........
第9个问题已回复..........
第10个问题已回复..........
第11个问题已回复..........
第12个问题已回复..........
第13个问题已回复..........
第14个问题已回复..........
第15个问题已回复..........
第16个问题已回复..........
第17个问题已回复..........
第18个问题已回复..........
第19个问题已回复..........
第20个问题已回复..........
第21个问题已回复..........
第22个问题已回复..........
第23个问题已回复..........
第24个问题已回复..........
第25个问题已回复..........
第26个问题已回复..........
第27个问题已回复..........
第28个问题已回复..........
第29个问题已回复..........
第30个问题已回复..........
第31个问题已回复..........
第32个问题已回复..........
第33个问题已回复..........
第34个问题已回复..........
第35个问题已回复..........
第36个问题已回复..........
第37个问题已回复..........
第38个问题已回复..........
第39个问题已回复..........
第40个问题已回复..........
第41个问题已回复..........
第42个问题已回复..........
第43个问题已回复..........
第44个问题已回复..........
第45个问题已回复..........
第46个问题已回复..........
第47个问题已回复..........
第48个问题已回复..........
第49个问题已回复..........
第50个问题已回复..........
第51个问题已回复

{'query_id': 125,
 'query': '125.电力现货技术支持系统与电力交易平台定位与关系是什么？',
 'gt_answer': '随着中发9号文的发布，我国电力市场改革不断深化，电力交易品种日益丰富，以中长期交易规避风险，以现货市场发现价格的“中长期+现货”联合运营模式逐渐形成。中发9号文配套文件《关于推进电力市场建设的实施意见》指出，电力市场主要由中长期市场和现货市场构成，中长期市场主要开展多年、年、季、月、周等日以上电能量交易和可中断负荷、调压等辅助服务交易。现货市场主要开展日前、日内、实施电能量交易和备用、调频等辅助服务交易。条件成熟时，探索开展容量市场、电力期货和衍生品等交易。2018年，国家电网公司《全国统一电力市场深化设计方案》在综合考虑体制架构和电网安全运行需要的基础上，提出了由交易中心负责市场注册、交易申报、交易结算、信息发布等与市场交易密切相关环节，由调控中心负责现货和辅助服务交易组织、安全校核等与电网运行密切相关环节的统一市场联合运营模式。此模式也成为了电力市场技术支撑平台在交易中心和调控中心之间功能建设的实际划分标准，交易中心的电力交易平台、调控中心的电力现货技术支持系统各功能相互协作实现“全时段、全过程、全环节”的交易流程，实现“中长期+现货”市场安全、高效、有序运营，向市场成员提供准确规范的市场交易信息和在线交易业务支持。调控中心负责现货交易与电网运行密切相关的环节，负责安全校核、现货交易组织、辅助服务交易组织等工作。电力现货技术支持系统面向调控中心业务，用于支撑日前、实时等电力现货市场的组织、出清、安全校核等核心业务，并与调控中心内部的调度系统、交易中心的电力交易平台等系统实现对接。交易中心负责与市场交易密切相关的环节，电力交易平台作为国网公司对市场主体服务的窗口，在市场注册、交易申报、交易结算、信息发布等环节为市场主体提供相关服务，并与包括电力现货技术支持系统在内的调度、营销、财务等系统实现对接。电力交易平台分为面向电力交易中心的主站端，以及面向发电侧、用电侧等市场成员的子站端。主站端的主要功能是市场成员注册审批、中长期交易组织、政府授权合约分解、合同管理、结算计算、信息发布、信息管理、信用管理等功能；子站端是发用侧市场成员用于进行市场注册、现货市场申报、中长期市场申报、中长期合约备案、查看发布信息、查看结算账单

In [16]:
import json

check_inputs["results"] = results
# 导出JSON文件
with open("电力现货市场QA.json", "w") as file:
    json.dump(check_inputs, file,indent=4, ensure_ascii=False)

### Step5 提问，运行QA链，得到RAG结果

In [ ]:
# question = "电力市场与普通商品市场的差异？"
# question = "什么是狭义的电力市场？"
# question = "电力市场有哪些特征？"
question = "输配电价对现货市场有什么影响？"
result = qa_chain({"query": question})
# result["result"]
print("========= chain result ==========")
print(result['result'])

In [ ]:
relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
    # print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
    print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}\nContent: {doc.page_content}")
    print("-"*100)
    print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')

### Step6 开始评估

In [ ]:
from ragchecker import RAGResults, RAGChecker
from ragchecker.metrics import all_metrics


# 从json/dict初始化ragresults
with open("examples/checking_inputs.json") as fp:
    rag_results = RAGResults.from_json(fp.read())

# 设置评估器
evaluator = RAGChecker(
    extractor_name="bedrock/meta.llama3-1-70b-instruct-v1:0",
    checker_name="bedrock/meta.llama3-1-70b-instruct-v1:0",
    batch_size_extractor=32,
    batch_size_checker=32
)

# 使用选定的指标（如retriever_metrics、generator_metrics、all_metrics）评估结果
evaluator.evaluate(rag_results, all_metrics)
print(rag_results)